In [ ]:
!wget -p https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip

--2021-10-23 01:35:50--  https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip
Resolving www.anatel.gov.br (www.anatel.gov.br)... 200.0.81.81, 2801:80:c90:c1da:da0:2312:2000:1331
Connecting to www.anatel.gov.br (www.anatel.gov.br)|200.0.81.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308415509 (294M) [application/x-zip-compressed]
Saving to: ‘www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip’

www.anatel.gov.br/d   5%[>                   ]  15.72M   755KB/s    in 23s     

2021-10-23 01:36:16 (701 KB/s) - Read error at byte 16482304/308415509 (Connection reset by peer). Retrying.

--2021-10-23 01:36:17--  (try: 2)  https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip
Connecting to www.anatel.gov.br (www.anatel.gov.br)|200.0.81.81|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 308415509 (294M), 291933205

In [ ]:
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile
import zipfile

In [ ]:
with zipfile.ZipFile('/content/www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip') as z:
  print(*z.namelist(), sep='\n')

Acessos_Banda_Larga_Fixa_2007-2010.csv
Acessos_Banda_Larga_Fixa_2007-2010_Colunas.csv
Acessos_Banda_Larga_Fixa_2011-2012.csv
Acessos_Banda_Larga_Fixa_2011-2012_Colunas.csv
Acessos_Banda_Larga_Fixa_2013-2014.csv
Acessos_Banda_Larga_Fixa_2013-2014_Colunas.csv
Acessos_Banda_Larga_Fixa_2015-2016.csv
Acessos_Banda_Larga_Fixa_2015-2016_Colunas.csv
Acessos_Banda_Larga_Fixa_2017-2018.csv
Acessos_Banda_Larga_Fixa_2017-2018_Colunas.csv
Acessos_Banda_Larga_Fixa_2019-2020.csv
Acessos_Banda_Larga_Fixa_2019-2020_Colunas.csv
Acessos_Banda_Larga_Fixa_2019_2020_Colunas.csv
Acessos_Banda_Larga_Fixa_2021.csv
Acessos_Banda_Larga_Fixa_2021_Colunas.csv
Acessos_Banda_Larga_Fixa_Total.csv
Densidade_Banda_Larga_Fixa.csv


In [ ]:
pasta = '/content/www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos'
broadband = os.path.join(pasta, 'acessos_banda_larga_fixa.zip')

In [ ]:
#decodificando os arquivos
  #Anatel separa os arquivos por biênio
  #Entre 2007 e 2010 os dados eram fornecidos por trimestre, a partir de 2011 os dados passam a ser mensais
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2007-2010.csv') as f:
    b_1 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2011-2012.csv') as f:
    b_2 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2013-2014.csv') as f:
    b_3 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2015-2016.csv') as f:
    b_4 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2017-2018.csv') as f:
    b_5 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2019-2020.csv') as f:
    b_6 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Acessos_Banda_Larga_Fixa_2021.csv') as f:
    b_7 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#junção das bases
bl = b_1.append([b_2, b_3, b_4, b_5, b_6, b_7], ignore_index=True)

In [ ]:
#renomeando variaveis
bl.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                 'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Município':'municipio',
                 'Código IBGE Município':'id_municipio', 'Faixa de Velocidade':'velocidade', 'Tecnologia':'tecnologia',
                 'Meio de Acesso':'transmissao', 'Acessos':'acessos', 'Tipo de Pessoa': 'pessoa'}, inplace=True)

In [ ]:
#organização das variáveis
bl.drop(['grupo_economico', 'municipio', 'pessoa'], axis=1, inplace=True)
bl = bl[['ano', 'mes', 'sigla_uf', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'transmissao', 'velocidade', 'acessos']]
bl.sort_values(['ano', 'mes', 'id_municipio'], inplace=True)

In [ ]:
#presença de variáveis
duplicado = bl.duplicated(subset=['ano', 'mes', 'sigla_uf', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'transmissao', 'velocidade']).any()
duplicado

True

In [ ]:
#os dados possuem duplicações mostrando diferentes resultados para acesso
#o agrupamento é feito pelas variáveis de identificação
bl['acessos_total'] = bl.groupby(['ano', 'mes', 'sigla_uf', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'transmissao', 'velocidade'])['acessos'].transform(np.sum)
#após ordenamento das observações, se mantém somente 1 linha que identifique as observações
bl.sort_values(['ano', 'mes', 'sigla_uf', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'transmissao', 'velocidade'], inplace=True)
bl.drop_duplicates(subset=['ano', 'mes', 'sigla_uf', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'transmissao', 'velocidade'], keep='first', inplace=True)
#exclui-se a coluna de acessos e renomeia
bl.drop('acessos', axis=1, inplace=True)
bl.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
#salvar arquivos no Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#exportação para csv
bl.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/banda_larga_microdados.csv', index=False, encoding="utf-8", na_rep="")

In [ ]:
#brasil
bl_br = bl.drop(['sigla_uf', 'id_municipio', 'empresa','cnpj', 'porte_empresa',  'tecnologia', 'transmissao', 'velocidade'], axis = 1)
bl_br['acessos_total'] = bl_br.groupby(['ano', 'mes'])['acessos'].transform(np.sum)
bl_br.sort_values(['ano', 'mes'], inplace=True)
bl_br.drop_duplicates(subset=['ano', 'mes'], keep ='first', inplace = True)
bl_br.drop('acessos', axis=1, inplace=True)
bl_br.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
#uf
bl_uf = bl.drop(['id_municipio', 'empresa','cnpj', 'porte_empresa',  'tecnologia', 'transmissao', 'velocidade'], axis = 1)
bl_uf['acessos_total'] = bl_uf.groupby(['ano', 'mes', 'sigla_uf'])['acessos'].transform(np.sum)
bl_uf.sort_values(['ano', 'mes', 'sigla_uf'], inplace=True)
bl_uf.drop_duplicates(subset=['ano', 'mes', 'sigla_uf'], keep ='first', inplace = True)
bl_uf.drop('acessos', axis=1, inplace=True)
bl_uf.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
#municipio
bl_mun = bl.drop(['empresa','cnpj', 'porte_empresa',  'tecnologia', 'transmissao', 'velocidade'], axis = 1)
bl_mun['acessos_total'] = bl_mun.groupby(['ano', 'mes', 'id_municipio'])['acessos'].transform(np.sum)
bl_mun.sort_values(['ano', 'mes', 'id_municipio'], inplace=True)
bl_mun.drop_duplicates(subset=['ano', 'mes', 'id_municipio'], keep ='first', inplace = True)
bl_mun.drop('acessos', axis=1, inplace=True)
bl_mun.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
with ZipFile(broadband) as z:
  with z.open(f'Densidade_Banda_Larga_Fixa.csv') as f:
    densidade = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
densidade.rename(columns={'Nível Geográfico Densidade':'Geografia'}, inplace=True)
densidade_br = densidade[densidade.Geografia.isin(['Brasil'])]
densidade_uf = densidade[densidade.Geografia.isin(['UF'])]
densidade_munic = densidade[densidade.Geografia.isin(['Municipio'])]

In [ ]:
densidade_br.drop(['UF', 'Município', 'Código IBGE', 'Geografia'], axis=1, inplace=True)
densidade_br.rename(columns={'Ano':'ano', 'Mês':'mes', 'Densidade':'densidade'}, inplace=True)
densidade_br['densidade'] = densidade_br['densidade'].apply(lambda x: str(x).replace(',','.'))
densidade_br = pd.merge(bl_br, densidade_br,
                         how='left', left_on=['ano', 'mes'],
                         right_on=['ano', 'mes'])
densidade_br['acessos'] = densidade_br['acessos'].astype('int64')
densidade_br.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/densidade_banda_larga_brasil.csv', index=False, encoding="utf-8", na_rep="")

In [ ]:
densidade_uf.drop(['Município', 'Código IBGE', 'Geografia'], axis=1, inplace=True)
densidade_uf.rename(columns={'Ano':'ano', 'Mês':'mes', 'UF':'sigla_uf', 'Densidade':'densidade'}, inplace=True)
densidade_uf['densidade'] = densidade_uf['densidade'].apply(lambda x: str(x).replace(',','.'))
densidade_uf = pd.merge(bl_uf, densidade_uf,
                         how='left', left_on=['ano', 'mes', 'sigla_uf'],
                         right_on=['ano', 'mes', 'sigla_uf'])
densidade_uf['acessos'] = densidade_uf['acessos'].astype('int64')
densidade_uf.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/densidade_banda_larga_uf.csv', index=False, encoding="utf-8", na_rep="")

In [ ]:
densidade_munic.drop(['Município', 'UF', 'Geografia'], axis=1, inplace=True)
densidade_munic.rename(columns={'Ano':'ano', 'Mês':'mes', 'Código IBGE':'id_municipio', 'Densidade':'densidade'}, inplace=True)
densidade_munic['densidade'] = densidade_munic['densidade'].apply(lambda x: str(x).replace(',','.'))
densidade_munic = pd.merge(bl_mun, densidade_munic,
                         how='left', left_on=['ano', 'mes', 'id_municipio'],
                         right_on=['ano', 'mes', 'id_municipio'])
densidade_munic['acessos'] = densidade_munic['acessos'].astype('int64')
densidade_mun.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/densidade_banda_larga_municipios.csv', index=False, encoding="utf-8", na_rep="")